In [5]:
from numpy import *
import operator
from os import listdir
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from numpy.linalg import *
from scipy.sparse.linalg import svds
from scipy.stats.stats import pearsonr
from numpy import linalg as la
import numpy as np

In [3]:
col = ['user_id','song_id','rating']
df = pd.read_csv('new_matrix_als.csv')
print(df)

df['user_id'] = df['user_id'].astype('category')
cat_columns = df.select_dtypes(['category']).columns
df[cat_columns] = df[cat_columns].apply(lambda x: x.cat.codes)
df['song_id'] = df['song_id'].astype('category')
cat_columns = df.select_dtypes(['category']).columns
df[cat_columns] = df[cat_columns].apply(lambda x: x.cat.codes)

rp = df.pivot_table(columns=['song_id'], index=['user_id'], values='rating')
rp = rp.fillna(0)

print(rp)

    user_id  song_id  rating
0         1        1       5
1         1        2       4
2         1        5       2
3         2        2       2
4         2        3       4
5         3        1       3
6         3        3       5
7         3        5       1
8         4        1       4
9         4        2       5
10        5        4       3
song_id    0    1    2    3    4
user_id                         
0        5.0  4.0  0.0  0.0  2.0
1        0.0  2.0  4.0  0.0  0.0
2        3.0  0.0  5.0  0.0  1.0
3        4.0  5.0  0.0  0.0  0.0
4        0.0  0.0  0.0  3.0  0.0


In [16]:
Q = rp.values
W = Q>0.5
W[W == True] = 1
W[W == False] = 0
# To be consistent with Q matrix
W = W.astype(np.float64, copy=False)

lambda_ = 0.1
n_factors = 3
m, n = Q.shape
n_iterations = 20

X = 5 * np.random.rand(m, n_factors) 
Y = 5 * np.random.rand(n_factors, n)

errors = []

In [17]:
print(Q)
print(W)
print(X)
print(Y)

[[5. 4. 0. 0. 2.]
 [0. 2. 4. 0. 0.]
 [3. 0. 5. 0. 1.]
 [4. 5. 0. 0. 0.]
 [0. 0. 0. 3. 0.]]
[[1. 1. 0. 0. 1.]
 [0. 1. 1. 0. 0.]
 [1. 0. 1. 0. 1.]
 [1. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0.]]
[[3.87619807 3.06881839 1.67913037]
 [0.24600693 1.7382197  3.04580121]
 [3.07860715 2.67568497 0.33907944]
 [0.85071199 1.96043683 1.6883778 ]
 [4.30570024 1.39002568 0.25950675]]
[[2.850506   0.58240033 3.135614   0.55333476 1.38317652]
 [4.44413765 3.74918615 2.49753664 2.07511135 3.83300981]
 [2.2893218  0.2385923  4.21257319 0.47326336 1.52013895]]


In [18]:
def get_error(Q, X, Y, W):
    return np.sum((W * (Q - np.dot(X, Y)))**2)

for ii in range(n_iterations):
    X = np.linalg.solve(np.dot(Y, Y.T) + lambda_ * np.eye(n_factors), np.dot(Y, Q.T)).T
    Y = np.linalg.solve(np.dot(X.T, X) + lambda_ * np.eye(n_factors), np.dot(X.T, Q))
    errors.append(get_error(Q, X, Y, W))
Q_hat = np.dot(X, Y)

print(Q_hat)

[[ 4.62031703  4.47190351  0.14114205 -0.21901073  1.06264632]
 [ 1.23182545  0.5818061   3.60674647  0.42371742  0.4282494 ]
 [ 2.068029    0.91475944  5.08778939 -0.28421235  0.73141518]
 [ 4.30777345  4.34499761 -0.1754079   0.22240185  0.95029647]
 [-0.35269443  0.36113373  0.09126932  2.76823175 -0.2407446 ]]


In [19]:
print(rp)

song_id    0    1    2    3    4
user_id                         
0        5.0  4.0  0.0  0.0  2.0
1        0.0  2.0  4.0  0.0  0.0
2        3.0  0.0  5.0  0.0  1.0
3        4.0  5.0  0.0  0.0  0.0
4        0.0  0.0  0.0  3.0  0.0
